# Clase: Toma de decisiones con IA

Ejemplo simple de **modelo de regresión** para mostrar en clases.

Escenario ficticio: *tiempo de espera promedio en un servicio de urgencias*.

## 1. Importar librerías

Usaremos `pandas`, `numpy` y un modelo de regresión lineal de `scikit-learn`.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score


## 2. Crear un dataset ficticio

Simularemos 400 observaciones de un servicio de urgencias con:
- `pacientes_en_sala`: número de pacientes esperando.
- `medicos_disponibles`: cantidad de médicos disponibles.
- `nivel_complejidad`: 1 = baja, 2 = media, 3 = alta.

La variable objetivo será `tiempo_espera_min`, en minutos.

In [ ]:
np.random.seed(42)

n = 400

pacientes_en_sala = np.random.randint(5, 60, size=n)
medicos_disponibles = np.random.randint(1, 8, size=n)
nivel_complejidad = np.random.choice([1, 2, 3], size=n, p=[0.4, 0.4, 0.2])

# Regla base para generar el tiempo de espera (ficticia)
tiempo_base = 10
efecto_pacientes = pacientes_en_sala * 1.2
efecto_medicos = -4 * medicos_disponibles
efecto_complejidad = np.where(nivel_complejidad == 1, 0,
                       np.where(nivel_complejidad == 2, 8, 20))

ruido = np.random.normal(0, 5, size=n)  # ruido aleatorio

tiempo_espera_min = tiempo_base + efecto_pacientes + efecto_medicos + efecto_complejidad + ruido
tiempo_espera_min = np.clip(tiempo_espera_min, 5, None)

df_urgencias = pd.DataFrame({
    "pacientes_en_sala": pacientes_en_sala,
    "medicos_disponibles": medicos_disponibles,
    "nivel_complejidad": nivel_complejidad,
    "tiempo_espera_min": tiempo_espera_min
})

df_urgencias.head()


## 3. Separar variables y objetivo

- X: características del sistema.
- y: tiempo de espera promedio en minutos.

In [ ]:
X = df_urgencias[["pacientes_en_sala", "medicos_disponibles", "nivel_complejidad"]]
y = df_urgencias["tiempo_espera_min"]

X.shape, y.shape


## 4. Dividir en entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("Tamaño entrenamiento:", X_train.shape[0])
print("Tamaño prueba:", X_test.shape[0])


## 5. Entrenar el modelo de regresión

In [ ]:
modelo_reg = LinearRegression()
modelo_reg.fit(X_train, y_train)

print("Modelo de regresión entrenado 👍")


## 6. Evaluar el modelo

Usaremos el **error absoluto medio (MAE)** y el **R²** para tener una idea de qué tan bien predice.

In [ ]:
y_pred = modelo_reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error absoluto medio (MAE): {mae:.2f} minutos")
print(f"R² (explicación de la variabilidad): {r2:.3f}")


## 7. Ver los coeficientes del modelo

Esto ayuda a explicar en clase el efecto de cada variable sobre el tiempo de espera.

In [ ]:
coeficientes = pd.DataFrame({
    "variable": X.columns,
    "coeficiente": modelo_reg.coef_
})

print("Intercepto (minutos base):", modelo_reg.intercept_)
coeficientes


## 8. Probar con escenarios ficticios

Podemos probar distintos escenarios para ver cómo varía el tiempo de espera:

In [ ]:
# Escenario 1: pocos pacientes, varios médicos, baja complejidad
escenario_1 = pd.DataFrame({
    "pacientes_en_sala": [10],
    "medicos_disponibles": [5],
    "nivel_complejidad": [1]
})

# Escenario 2: muchos pacientes, pocos médicos, alta complejidad
escenario_2 = pd.DataFrame({
    "pacientes_en_sala": [50],
    "medicos_disponibles": [2],
    "nivel_complejidad": [3]
})

print("Escenario 1 - Tiempo de espera estimado (min):", modelo_reg.predict(escenario_1)[0])
print("Escenario 2 - Tiempo de espera estimado (min):", modelo_reg.predict(escenario_2)[0])


## 9. Resumen para la clase

- Este modelo de regresión **predice un número**: el tiempo de espera.
- Usa como insumo variables operacionales: cantidad de pacientes, médicos y complejidad.
- Sirve para **planificar recursos**, simular escenarios y tomar decisiones informadas.

La lógica es la misma que en muchos problemas reales de gestión pública y de servicios: tiempos de respuesta, duración de hospitalizaciones, montos de subsidios, demanda de atención, etc.